In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score,mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv("Laptops.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/laptop-details/Laptops.csv'

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
df.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
df['Operating System'].value_counts()

In [ ]:
def os(data):
    if 'windows' in data.lower():
        return "Windows"
    elif 'mac' in data.lower():
        return "Mac"
    else:
        return "Others"

df['Operating System'] = df['Operating System'].apply(os)


In [ ]:
df["Processor"] = df['Processor'].apply(lambda x:" ".join(x.split()[0:2]))

In [ ]:
df['Processor'].value_counts()

In [ ]:
def processor(text):
    if "Celeron" in text:
        return "Celeron"
    elif "Core" in text:
        if "9" in text:
            return "Others"
        else:
            return text
    elif "Ryzen" in text:
        if any(m in text for m in ['Z1', "9"]):
            return "Others"
        else:
            return text
    elif any(m in text for m in ["M1", "M2", "M3"]):
        return "Mac"
    else:
        return "Others"

df["Processor"] = df['Processor'].apply(processor)

In [ ]:
df.Processor.value_counts()

In [ ]:
df['Storage'] = df['Storage'].str.replace("GB","")
df['Storage'] = df['Storage'].str.replace("TB","000")
df['Storage'] = df['Storage'].str.replace(" ","")
df.sample(5)

In [ ]:
df['RAM'] = df['RAM'].str.replace("GB","")
df.sample(5)

In [ ]:
df['Screen Size'].value_counts()

In [ ]:
df['Screen Size'] = df['Screen Size'].apply(lambda x:"".join(x.split(" ")[0]))

In [ ]:
df.Price = df.Price.str.replace("₹","")
df.Price = df.Price.str.replace(",","")
df.sample(5)

In [ ]:
df[['Storage','RAM','Price']] = df[['Storage','RAM','Price']].astype(int)

In [ ]:
df["Screen Size"] = df['Screen Size'].astype(float)

In [ ]:
df.info()

In [ ]:
df.drop(columns="Model Name",inplace=True)

In [ ]:
df.sample()

In [ ]:
sns.heatmap(df.corr(numeric_only=True))

In [ ]:
sns.distplot(df.Price)

In [ ]:
df['Processor'].value_counts().plot(kind = 'bar')

In [ ]:
df['Brand'].value_counts()

In [ ]:
rare_b = ['SAMSUNG','GIGABYTE','CHUWI','WINGS','ZEBRONICS','Ultimus','LG','realme','Primebook','MICROSOFT']
df.Brand = df.Brand.apply(lambda x: "Others" if x in rare_b else x)

In [ ]:
df['Brand'].value_counts().plot(kind = 'bar')

In [ ]:
sns.barplot(x=df.Brand,y=df.Price)
plt.xticks(rotation = 'vertical')
plt.show()

In [ ]:
sns.barplot(x=df['Operating System'],y=df.Price)
plt.xticks(rotation = "vertical")
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
X= df.drop(columns="Price")
Y = np.log(df['Price'])
sns.distplot(Y)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=83)

In [ ]:
X_train

In [ ]:
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first'),[0,1,2,6])
],remainder='passthrough')
step_2  = LinearRegression()
pipe = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first',handle_unknown="ignore"),[0,1,2,6])
],remainder='passthrough')
step_2  = RandomForestRegressor()
pipe_rf = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe_rf.fit(X_train,Y_train)
y_pred = pipe_rf.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first'),[0,1,2,6])
],remainder='passthrough')
step_2  = KNeighborsRegressor(n_neighbors=3)
pipe = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
from sklearn.tree import DecisionTreeRegressor
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first'),[0,1,2,6])
],remainder='passthrough')
step_2  = DecisionTreeRegressor()
pipe = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
from xgboost import XGBRegressor
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=100)
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first'),[0,1,2,6])
],remainder='passthrough')
step_2 = XGBRegressor(n_estimators=45,max_depth=5,learning_rate=0.5)
pipe = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first',handle_unknown='infrequent_if_exist'),[0,1,2,6])
],remainder='passthrough')
step_2 = GradientBoostingRegressor(n_estimators=500,max_depth=15,max_features=0.75,)
pipe = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))

print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
def best_rs(X,Y):
    r2_score_ =[]
    try:
        for i in range(1,100):
            X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=i)
            pipe_rf.fit(X_train,Y_train)
            y_pred = pipe_rf.predict(X_test)
            r2_score_.append(r2_score(Y_test,y_pred))
        return r2_score_
    except ValueError as e:
        print("error")

In [ ]:
import warnings
warnings.simplefilter("ignore") 

In [ ]:
result = best_rs(X,Y)
print(np.argmax(result))

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=84)
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first',handle_unknown="ignore"),[0,1,2,6])
],remainder='passthrough')
step_2  = RandomForestRegressor()
pipe_rf = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe_rf.fit(X_train,Y_train)
y_pred = pipe_rf.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))

In [ ]:
np.random.seed(42)
from sklearn.model_selection import GridSearchCV
param_dist = {'step2__n_estimators': [50,100,150,200,500],
              'step2__max_depth': [2, 3, 4, 8, 15,None],
              
              'step2__max_samples': [0.5, 0.75, 1.0],
              'step2__max_features': ['sqrt', 'log2', None],
              'step2__criterion': ["squared_error", "absolute_error"]}

cv_rf = GridSearchCV(estimator=pipe_rf, 
                     cv = 5,
                     param_grid=param_dist,
                     n_jobs = -1)

cv_rf.fit(X,Y)

print('Best Parameters using grid search: \n',
      cv_rf.best_params_)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=84)
step_1 = ColumnTransformer(transformers=[
    ('col_tnf', OneHotEncoder(sparse_output=False, drop='first',handle_unknown='infrequent_if_exist'),[0,1,2,6])
],remainder='passthrough')
step_2  = RandomForestRegressor(n_estimators=500,
                                max_samples=1.0,
                                max_features='log2',
                                max_depth=15,
                                criterion='squared_error'
)
pipe_rf = Pipeline([('step1',step_1),
                ('step2',step_2)])
pipe_rf.fit(X_train,Y_train)
y_pred = pipe_rf.predict(X_test)

print('R2 score',r2_score(Y_test,y_pred))
print('MAE',mean_absolute_error(Y_test,y_pred))


In [ ]:
query = np.array(['Apple','Mac','Mac',256,8,39.23,'No'])

query = query.reshape(1,7)
np.exp(pipe_rf.predict(query))

In [ ]:
import pickle
with open('laptop_price_model.pkl', 'wb') as f:
    pickle.dump(pipe_rf, f)